## Basic Message Routing (Format Based)

This notebook demonstrates simple routing, picking up messages from one topic and writing it to different topic based on the message format.

Simple routing allows filtering using:
- origin_sender
- origin_topic
- origin_message_format

In this case, we are routing based on origin message format.

[ProbeAgent] --_MessageType1_-> (default_topic) -> [Routing Agent/RoutingAgent:wire_message] -> (**topic_1**)

[ProbeAgent] --_MessageType2_-> (default_topic) -> [Routing Agent/RoutingAgent:wire_message] -> (**topic_2**)


In [1]:
# Define different message types

from pydantic import BaseModel

class MessageType1(BaseModel):
    content: str

class MessageType2(BaseModel):
    content: str
    additional_info: str

In [2]:
# Create a wiring agent

from rustic_ai.core.agents.eip.basic_wiring_agent import BasicWiringAgent
from rustic_ai.core.guild.builders import AgentBuilder, GuildBuilder, RouteBuilder
from rustic_ai.core.utils.basic_class_utils import get_qualified_class_name

routing_agent = (
    AgentBuilder(BasicWiringAgent)
    .set_id("RoutingAgent")
    .set_name("Routing Agent")
    .set_description("A simple message router that routes messages based on their type.")
    .build_spec()
)

# Create the Routing Rule for MessageType1

routing_rule_one = (
    RouteBuilder(routing_agent)
    .filter_on_origin(origin_message_format=get_qualified_class_name(MessageType1))
    .set_destination_topics("topic_1")
    .build()
)

# Create the Routing Rule for MessageType2
routing_rule_two = (
    RouteBuilder(routing_agent)
    .filter_on_origin(origin_message_format=get_qualified_class_name(MessageType2))
    .set_destination_topics("topic_2")
    .build()
)

# Create the Guild with the Routing Agent and the Routing Rules
guild = (
    GuildBuilder(
        guild_id="MessageRouterGuild",
        guild_name="Message Router Guild",
        guild_description="A guild that routes messages based on their type.",
    )
    .add_agent_spec(routing_agent)
    .add_route(routing_rule_one)
    .add_route(routing_rule_two)
    .launch("myorg")
)

In [3]:
from rustic_ai.core.agents.testutils.probe_agent import ProbeAgent

probe_agent = (
    AgentBuilder(ProbeAgent)
    .set_id("ProbeAgent")
    .set_name("Probe Agent")
    .set_description("A probe agent to test the routing of messages.")
    .add_additional_topic("topic_1")
    .add_additional_topic("topic_2")
    .build()
)

guild._add_local_agent(probe_agent)

In [ ]:
from rustic_ai.core.guild.dsl import GuildTopics

guild_default_topic = GuildTopics.DEFAULT_TOPICS[0]

probe_agent.publish_with_guild_route(
    payload=MessageType1(content="Hello from MessageType1!"),
    topic=guild_default_topic,
)

In [5]:
probe_agent.print_message_history()

(default_topic) -> [Routing Agent/RoutingAgent:wire_message] -> (topic_1)


In [6]:
probe_agent.publish_with_guild_route(
    payload=MessageType2(content="Hello from MessageType2!", additional_info="Some extra info"),
    topic=guild_default_topic,
)

In [7]:
probe_agent.print_message_history()

(default_topic) -> [Routing Agent/RoutingAgent:wire_message] -> (topic_2)
